In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import gc
import random

import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings(action='ignore')

from pycaret.classification import *

import random as python_random
np.random.seed(42)
python_random.seed(42)
set_config('seed', 42)

In [2]:
PATH = './data/'

train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qua = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qua  = pd.read_csv(PATH+'test_quality_data.csv')

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
train_err

In [3]:
train_err['date'] = train_err['time'].astype(str).str.slice(0,8)
train_err['model_fwver'] = train_err.model_nm + train_err.fwver
train_err['errtype_code'] = train_err.errtype.astype(str) + train_err.errcode

train_err_0 = train_err[['user_id','date','model_fwver','errtype_code']].drop_duplicates()
display(train_err_0.head())

test_err['date'] = test_err['time'].astype(str).str.slice(0,8)
test_err['model_fwver'] = test_err.model_nm + test_err.fwver
test_err['errtype_code'] = test_err.errtype.astype(str) + test_err.errcode

test_err_0 = test_err[['user_id','date','model_fwver','errtype_code']].drop_duplicates()
display(test_err_0.head())

,user_id,date,model_fwver,errtype_code
0,10000,20201101,model_305.15.2138,151
1,10000,20201101,model_305.15.2138,121
2,10000,20201101,model_305.15.2138,111
3,10000,20201101,model_305.15.2138,161
4,10000,20201101,model_305.15.2138,40


,user_id,date,model_fwver,errtype_code
0,30000,20201101,model_104.16.3553,311
1,30000,20201101,model_104.16.3553,332
2,30000,20201101,model_104.16.3553,151
3,30000,20201101,model_104.16.3553,221
4,30000,20201101,model_104.16.3553,111


In [ ]:
train_err.shape, train_err_0.shape

In [4]:
train_err_1 = train_err.groupby(['user_id','date']).count().reset_index()[['user_id','date','time']]
train_err_1.columns = ['user_id','date','date_cnt']
train_err_1 = train_err_1.groupby('user_id').agg({'date':'count','date_cnt': 'sum'}).reset_index()
train_err_1.columns = ['user_id','date_cnt', 'date_sum']
display(train_err_1.head())

test_err_1 = test_err.groupby(['user_id','date']).count().reset_index()[['user_id','date','time']]
test_err_1.columns = ['user_id','date','date_cnt']
test_err_1 = test_err_1.groupby('user_id').agg({'date':'count','date_cnt': 'sum'}).reset_index()
test_err_1.columns = ['user_id','date_cnt', 'date_sum']
display(test_err_1.head())

train_err_1.shape

,user_id,date_cnt,date_sum
0,10000,30,317
1,10001,30,2365
2,10002,29,306
3,10003,30,306
4,10004,30,777


,user_id,date_cnt,date_sum
0,30000,29,2750
1,30001,28,284
2,30002,30,941
3,30003,28,371
4,30004,30,881


(15000, 3)

In [5]:
train = train_err_1.copy()
test = test_err_1.copy()

In [6]:
train_model = set(train_err.model_fwver.unique())
test_model = set(test_err.model_fwver.unique())
models = train_model & test_model
models = list(models)

In [7]:
for model in models:
    train[model] = 0
    test[model] = 0
    
train.shape, test.shape    

((15000, 34), (14998, 34))

In [ ]:
train

In [8]:
train_err_2 = train_err.groupby(['user_id','model_fwver']).count().reset_index()[['user_id','model_fwver','time']]
train_err_2.columns = ['user_id','model_fwver','model_fwver_cnt']
#train_err_2['model_fwver_rank'] = train_err_2.model_fwver_cnt.rank()
display(train_err_2.head())

test_err_2 = test_err.groupby(['user_id','model_fwver']).count().reset_index()[['user_id','model_fwver','time']]
test_err_2.columns = ['user_id','model_fwver','model_fwver_cnt']
#test_err_2['model_fwver_rank'] = test_err_2.model_fwver_cnt.rank()
display(test_err_2.head())

,user_id,model_fwver,model_fwver_cnt
0,10000,model_305.15.2138,317
1,10001,model_204.33.1185,379
2,10001,model_204.33.1261,1986
3,10002,model_305.15.2138,306
4,10003,model_204.33.1185,81


,user_id,model_fwver,model_fwver_cnt
0,30000,model_104.16.3553,2320
1,30000,model_104.16.3571,367
2,30000,model_204.33.1261,63
3,30001,model_305.15.2138,284
4,30002,model_004.22.1750,733


In [9]:
for idx, col in tqdm(enumerate(train_err_2.values)):
    i = train[train.user_id == train_err_2.loc[idx].user_id].index
    c = train.columns[train.columns == train_err_2.loc[idx].model_fwver]
    
    train.loc[i, c] += train_err_2.loc[idx].model_fwver_cnt    

for idx, col in tqdm(enumerate(test_err_2.values)):
    i = test[test.user_id == test_err_2.loc[idx].user_id].index
    c = test.columns[test.columns == test_err_2.loc[idx].model_fwver]
    
    test.loc[i, c] += test_err_2.loc[idx].model_fwver_cnt    

24062it [01:08, 352.33it/s]
24096it [01:08, 352.25it/s]


In [10]:
train_error = set(train_err.errtype.unique())
test_error = set(test_err.errtype.unique())
errors = train_error & test_error
errors = list(errors)

In [11]:
for error in errors:
    train[error] = 0
    test[error] = 0
    
train.shape, test.shape    

((15000, 75), (14998, 75))

In [12]:
# train_err > errtype => rank
train_err_3 = train_err.groupby(['user_id','errtype']).count().reset_index()[['user_id','errtype','time']]
train_err_3.columns = ['user_id','errtype','errtype_cnt']
#train_err_3['errtype_rank'] = train_err_3.errtype_cnt.rank()
display(train_err_3.head())

# test_err > errtype => rank
test_err_3 = test_err.groupby(['user_id','errtype']).count().reset_index()[['user_id','errtype','time']]
test_err_3.columns = ['user_id','errtype','errtype_cnt']
#test_err_3['errtype_rank'] = test_err_3.errtype_cnt.rank()
display(test_err_3.head())

,user_id,errtype,errtype_cnt
0,10000,3,8
1,10000,4,104
2,10000,6,1
3,10000,7,1
4,10000,10,7


,user_id,errtype,errtype_cnt
0,30000,5,62
1,30000,6,1
2,30000,7,1
3,30000,11,16
4,30000,12,16


In [13]:
for idx, col in tqdm(enumerate(train_err_3.values)):
    i = train[train.user_id == train_err_3.loc[idx].user_id].index
    c = train.columns[train.columns == train_err_3.loc[idx].errtype]
    
    train.loc[i, c] += train_err_3.loc[idx].errtype_cnt    

for idx, col in tqdm(enumerate(test_err_3.values)):
    i = test[test.user_id == test_err_3.loc[idx].user_id].index
    c = test.columns[test.columns == test_err_3.loc[idx].errtype]
    
    test.loc[i, c] += test_err_3.loc[idx].errtype_cnt    

231039it [11:33, 333.33it/s]
231241it [11:16, 341.94it/s]


In [ ]:
train

In [ ]:
# train_err > errtype_code => rank
train_err_4 = train_err.groupby(['model_fwver','errtype_code']).count().reset_index()[['model_fwver','errtype_code','user_id']]
train_err_4.columns = ['model_fwver','errtype_code','cnt']
train_err_4['rank'] = train_err_4.cnt.rank()
display(train_err_4.head())

In [ ]:
# test_err > errtype_code => rank
test_err_4 = test_err.groupby(['model_fwver','errtype_code']).count().reset_index()[['model_fwver','errtype_code','user_id']]
test_err_4.columns = ['model_fwver','errtype_code','cnt']
test_err_4['rank'] = test_err_4.cnt.rank()
display(test_err_4.head())

In [ ]:
train = train_err.merge(train_err_1, on=['user_id'], how='left').merge(train_err_4, on=['model_fwver','errtype_code'], how='left')#.merge(train_err_3, on='errtype_code', how='left')
train = train[['user_id','date_cnt','date_sum','model_fwver','errtype_code','cnt','rank']].drop_duplicates()
train = train.groupby(['user_id','date_cnt','date_sum']).agg({'errtype_code':['count'],'cnt':['sum'],'rank':['max']}).reset_index()
train.columns = ['user_id','date_cnt','date_sum','errtype_code_cnt','cnt_sum','rank_max']
train

In [ ]:
test = test_err.merge(test_err_1, on=['user_id'], how='left').merge(test_err_4, on=['model_fwver','errtype_code'], how='left')#.merge(test_err_3, on='errtype_code', how='left')
test = test[['user_id','date_cnt','date_sum','model_fwver','errtype_code','cnt','rank']].drop_duplicates()
test = test.groupby(['user_id','date_cnt','date_sum']).agg({'errtype_code':['count'],'cnt':['sum'],'rank':['max']}).reset_index()
test.columns = ['user_id','date_cnt','date_sum','errtype_code_cnt','cnt_sum','rank_max']
test

In [14]:
train_qua_0 = train_qua.drop_duplicates()
test_qua_0 = test_qua.drop_duplicates()

train_qua_0.drop(['quality_3','quality_4','time','fwver'], axis=1, inplace=True)
test_qua_0.drop(['quality_3','quality_4','time','fwver'], axis=1, inplace=True)

train_qua_0.fillna(0, inplace=True)
test_qua_0.fillna(0, inplace=True)

In [15]:
def chg_qua(x):
    if x == 0:
        return 0
    else:
        return 1

In [16]:
cols = ['quality_0', 'quality_1', 'quality_2', 'quality_5','quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10','quality_11', 'quality_12']

for col in cols:
    train_qua_0[col] = train_qua_0[col].apply(chg_qua)
    test_qua_0[col] = test_qua_0[col].apply(chg_qua)

In [17]:
train_qua_1 = train_qua_0.groupby('user_id').sum().reset_index()
test_qua_1 = test_qua_0.groupby('user_id').sum().reset_index()

In [ ]:
train_qua_1

In [18]:
%%time
for col in cols:
    train_qua_1[col] = train_qua_1[col].apply(chg_qua)
    test_qua_1[col] = test_qua_1[col].apply(chg_qua)

Wall time: 62.9 ms


In [ ]:
cols=['quality_0', 'quality_1','quality_2','quality_5',  'quality_6','quality_7', 'quality_10',]

In [ ]:
train_qua_1.drop(cols, axis=1, inplace=True)
test_qua_1.drop(cols, axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(train_qua_1.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(test_qua_1.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [19]:
train = train.merge(train_qua_1, on='user_id', how='left').fillna(0)
test = test.merge(test_qua_1, on='user_id', how='left').fillna(0)

In [20]:
train_prob1 = train_prob.groupby('user_id').count().reset_index()
train_prob1.time = train_prob1.time.apply(chg_qua)
train_prob1.columns = ['user_id', 'prob']
train_prob1

,user_id,prob
0,10001,1
1,10004,1
2,10005,1
3,10006,1
4,10008,1
...,...,...
4995,24983,1
4996,24984,1
4997,24990,1
4998,24997,1


In [21]:
train = train.merge(train_prob1, on='user_id', how='left').fillna(0)

In [22]:
display(train.head())
display(test.head())

,user_id,date_cnt,date_sum,model_305.15.2092,model_804.73.2571,model_204.33.1149,model_403.11.1167,model_610,model_004.22.1656,model_305.15.2138,...,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,prob
0,10000,30,317,0,0,0,0,0,0,317,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1,10001,30,2365,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10002,29,306,0,0,0,0,0,0,306,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
3,10003,30,306,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,30,777,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0


,user_id,date_cnt,date_sum,model_305.15.2092,model_804.73.2571,model_204.33.1149,model_403.11.1167,model_610,model_004.22.1656,model_305.15.2138,...,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,29,2750,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
1,30001,28,284,0,0,0,0,0,0,284,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2,30002,30,941,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
3,30003,28,371,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
4,30004,30,881,0,0,3,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
train.shape, test.shape

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(train.corr(), annot=True, fmt='.2f', cmap='RdYlGn_r')

In [ ]:
test.columns

In [ ]:
cols = ['date_cnt', 'date_sum', 'model_fwver_cnt', 'model_fwver_max','errtype_code_max']

In [ ]:
min = train[cols].min()
max  = train[cols].max()

In [ ]:
for i, col in enumerate(cols):
    train[col] = (train[col] - min[i]) / (max[i] - min[i])
    test[col] = (test[col] - min[i]) / (max[i] - min[i])

In [23]:
train.prob = train.prob.astype(int)

In [24]:
cols = ['quality_0', 'quality_1', 'quality_2', 'quality_5','quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10','quality_11', 'quality_12']
train[cols] = train[cols].astype(int)
test[cols] = test[cols].astype(int)

In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 87 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   user_id            15000 non-null  int64
 1   date_cnt           15000 non-null  int64
 2   date_sum           15000 non-null  int64
 3   model_305.15.2092  15000 non-null  int64
 4   model_804.73.2571  15000 non-null  int64
 5   model_204.33.1149  15000 non-null  int64
 6   model_403.11.1167  15000 non-null  int64
 7   model_610          15000 non-null  int64
 8   model_004.22.1656  15000 non-null  int64
 9   model_305.15.2138  15000 non-null  int64
 10  model_204.33.1171  15000 non-null  int64
 11  model_004.22.1750  15000 non-null  int64
 12  model_204.33.1185  15000 non-null  int64
 13  model_68.5.3       15000 non-null  int64
 14  model_504.82.1684  15000 non-null  int64
 15  model_104.16.3569  15000 non-null  int64
 16  model_104.16.3571  15000 non-null  int64
 17  model_004.22

In [33]:
train.columns[1:86]

Index(['date_cnt', 'date_sum', 'model_305.15.2092', 'model_804.73.2571',
       'model_204.33.1149', 'model_403.11.1167', 'model_610',
       'model_004.22.1656', 'model_305.15.2138', 'model_204.33.1171',
       'model_004.22.1750', 'model_204.33.1185', 'model_68.5.3',
       'model_504.82.1684', 'model_104.16.3569', 'model_104.16.3571',
       'model_004.22.1666', 'model_305.15.2114', 'model_504.82.1730',
       'model_104.16.3553', 'model_004.22.1778', 'model_204.33.1261',
       'model_804.73.2237', 'model_305.15.3104', 'model_705.66.3237',
       'model_504.82.1778', 'model_305.15.2120', 'model_705.66.3571',
       'model_104.16.3439', 'model_004.22.1684', 'model_403.11.1149',
       'model_403.11.1141', 'model_204.33.1125', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '30', '31',
       '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
   

In [34]:
%%time
clf = setup(session_id=42, data=train, target='prob'
           , numeric_features=train.columns[1:86])

Setup Succesfully Completed!


,Description,Value
0,session_id,42
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(15000, 87)"
4,Missing Values,False
5,Numeric Features,86
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


Wall time: 4.98 s


In [35]:
best = compare_models(sort = 'AUC', n_select = 5
                     ,exclude=['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'ridge','qda']
                     )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.7945,0.8147,0.5014,0.8098,0.6190,0.4888,0.5153,10.8134
1,Gradient Boosting Classifier,0.7865,0.8110,0.4674,0.8125,0.5931,0.4624,0.4950,3.1692
2,Light Gradient Boosting Machine,0.7897,0.8090,0.5060,0.7868,0.6157,0.4803,0.5025,0.4846
3,Ada Boost Classifier,0.7750,0.7932,0.4797,0.7567,0.5869,0.4428,0.4647,0.9042
4,Extra Trees Classifier,0.7797,0.7932,0.4906,0.7640,0.5974,0.4555,0.4768,0.5831
5,Extreme Gradient Boosting,0.7783,0.7904,0.5266,0.7329,0.6126,0.4632,0.4757,1.7882
6,Random Forest Classifier,0.7635,0.7748,0.4557,0.7345,0.5623,0.4122,0.4344,0.1208
7,Linear Discriminant Analysis,0.7507,0.7428,0.3420,0.7924,0.4776,0.3463,0.3991,0.1518


In [36]:
blended = blend_models(estimator_list = best, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7843,0.7988,0.4743,0.7962,0.5944,0.4601,0.4887
1,0.8038,0.8280,0.5186,0.8288,0.6380,0.5130,0.5395
2,0.7895,0.8158,0.4829,0.8086,0.6047,0.4734,0.5026
3,0.7952,0.8250,0.4786,0.8375,0.6091,0.4840,0.5188
4,0.7813,0.7969,0.4786,0.7809,0.5934,0.4554,0.4810
Mean,0.7908,0.8129,0.4866,0.8104,0.6079,0.4772,0.5061
SD,0.0080,0.0130,0.0162,0.0207,0.0162,0.0205,0.0211


In [37]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7883,0.8116,0.478,0.8083,0.6008,0.4693,0.4993


In [38]:
%%time
final_model = finalize_model(blended)

Wall time: 3min 7s


In [39]:
test_x = sample_submssion[['user_id']].merge(test, on='user_id', how='left').fillna(0)
test_x.shape

(14999, 86)

In [40]:
predictions = predict_model(final_model, data = test_x)

In [41]:
sample_submssion['problem'] = predictions['Score']
sample_submssion.head()

,user_id,problem
0,30000,0.8180
1,30001,0.2446
2,30002,0.3109
3,30003,0.7792
4,30004,0.7796


In [42]:
sample_submssion.to_csv("./submission/submission_20210128-3.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.8180
1,30001,0.2446
2,30002,0.3109
3,30003,0.7792
4,30004,0.7796
...,...,...
14994,44994,0.3646
14995,44995,0.3177
14996,44996,0.5378
14997,44997,0.8017
